<a href="https://colab.research.google.com/github/fellowship/deep-and-wide-bandit/blob/dev/EDA_LargeSet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install -U ipykernel
#!pip install modin[dask]

In [2]:
%load_ext autoreload
%autoreload 2
%env JOBLIB_TEMP_FOLDER=/tmp

env: JOBLIB_TEMP_FOLDER=/tmp


In [3]:
import sys

!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

Requirement already up-to-date: pandas-profiling[notebook] in /usr/local/lib/python3.6/dist-packages (2.10.0)
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# Mount Drive

In [4]:
from google.colab import drive
drive.mount("/content/GDrive")

Mounted at /content/GDrive


In [5]:
import os
import pandas as pd
from pandas_profiling import ProfileReport
from zipfile import ZipFile

#Formatting .describe() calls s.t. floats are displayed in non-scientific notation
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [6]:
os.chdir(os.getcwd())
zip_path_l = ["/content/GDrive/MyDrive/July_Dec_2019.zip",
               "/content/GDrive/MyDrive/Jan_June_2020.zip", 
               "/content/GDrive/MyDrive/Jul_Sep_2020.zip"]

folder_name_l = [os.path.split(zip_path)[-1][:-4] for zip_path in zip_path_l]

for (zip_path, folder_name) in zip(zip_path_l, folder_name_l):
  
  if not(os.path.exists(folder_name)):
    
    with ZipFile(zip_path, 'r') as zip_obj:

      zip_obj.extractall(folder_name)

In [11]:
file_name_l = ['/content/July_Dec_2019/sends_july_dec_2019_2000',
               '/content/Jan_June_2020/sends_jan_june_2020_2000',
               '/content/Jul_Sep_2020/sends_july_sept_2020_2000']

col_names = ['riid','send_dt','launch_id','sub_sourcedev','opened','unsub',
             'rev_3dv2','aq_dt','aq_mo','aq_dow','aq_period','sends_since_last_open',
             'wk1_opens','wk4_opens','mo3_opens','mo6_opens','yr1_opens',
             'wk1_clicks','wk4_clicks','mo3_clicks','mo6_clicks',
             'rev_dec','rev_raw','snds_dec','snds_opens_dec','wk1_vis',
             'wk4_vis','mo3_vis','mo6_vis','wk1_pv','wk4_pv','mo3_pv','mo6_pv',
             'wk1_secs','wk4_secs','mo3_secs','mo6_secs','prev_optouts',
             'last_order','last_click','last_cart','last_visit','last_ded_open',
             'sent_time','offer_signature_id','dynamic_content_signature_id',
             'message_size','open_cnt','first_open_time','last_open_time',
             'hrs_to_first_open','hrs_to_last_open','click_cnt','first_click_time',
             'last_click_time','offer_category','hrs_to_first_click','hrs_to_last_click',
             'opt_out_cnt','first_opt_out_time','last_opt_out_time','opt_out_source',
             'opt_out_reason','hrs_to_first_opt_out','hrs_to_last_opt_out','campaign_id',
             'subject','marketing_strategy','campaign_type']

#os.remove('/content/July_Dec_2019/sends_july_dec_2019_2000')
#df2_jan_jun_20 = pd.read_csv(file_name_l[1], sep='\t', header=None, names=col_names)
#os.remove('/content/Jan_June_2020/sends_jan_june_2020_2000')
#df3_jul_sep_20 = pd.read_csv(file_name_l[2], sep='\t', header=None, names=col_names)
#os.remove('/content/Jul_Sep_2020/sends_july_sept_2020_2000')

In [13]:
df_1 = pd.DataFrame()
df_10 = pd.DataFrame()
df_50 = pd.DataFrame()
df_100 = pd.DataFrame()
cnt = 0
reader = pd.read_csv(file_name_l[0], sep='\t', header=None, names=col_names, chunksize=1e5)

for cnt in range(0, 101):  
  
  if not((cnt+1) % 10):
    print(f"[INFO] Operating on chunk #{cnt+1}")
  
  chunk = next(reader)
  if cnt in [0, 9, 49, 99]:
    id = "df_" + str(cnt+1)
    print(f"[INFO] Initializing the dataframe {id}:")
    globals()[id] = globals()[id].append(chunk)

[INFO] Initializing the dataframe df_1:
[INFO] Operating on chunk #10
[INFO] Initializing the dataframe df_10:
[INFO] Operating on chunk #20
[INFO] Operating on chunk #30
[INFO] Operating on chunk #40
[INFO] Operating on chunk #50
[INFO] Initializing the dataframe df_50:
[INFO] Operating on chunk #60
[INFO] Operating on chunk #70
[INFO] Operating on chunk #80
[INFO] Operating on chunk #90


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (59,60,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


[INFO] Operating on chunk #100
[INFO] Initializing the dataframe df_100:


In [ ]:
df_1.info()

In [ ]:
print(df_1.head(10))

In [10]:
df_1.head(10)

,riid,send_dt,launch_id,sub_sourcedev,opened,unsub,rev_3dv2,aq_dt,aq_mo,aq_dow,aq_period,sends_since_last_open,wk1_opens,wk4_opens,mo3_opens,mo6_opens,yr1_opens,wk1_clicks,wk4_clicks,mo3_clicks,mo6_clicks,rev_dec,rev_raw,snds_dec,snds_opens_dec,wk1_vis,wk4_vis,mo3_vis,mo6_vis,wk1_pv,wk4_pv,mo3_pv,mo6_pv,wk1_secs,wk4_secs,mo3_secs,mo6_secs,prev_optouts,last_order,last_click,last_cart,last_visit,last_ded_open,sent_time,offer_signature_id,dynamic_content_signature_id,message_size,open_cnt,first_open_time,last_open_time,hrs_to_first_open,hrs_to_last_open,click_cnt,first_click_time,last_click_time,offer_category,hrs_to_first_click,hrs_to_last_click,opt_out_cnt,first_opt_out_time,last_opt_out_time,opt_out_source,opt_out_reason,hrs_to_first_opt_out,hrs_to_last_opt_out,campaign_id,subject,marketing_strategy,campaign_type
0,251392782,2019-08-11,175407402,Checkout_Online,1,0,48.98000,379,7,6,Non-Holiday,16,0,0,1,4,9,0,0,0,1,0.19021,128.97000,6.49108,0.22946,0,0,0,0,0,0,0,0,0,0,0,0,0,378.00000,175.00000,nan,nan,35.00000,2019-08-11 17:11:15,581429320,nan,149211,1.00000,2019-08-11 19:34:58,2019-08-11 19:34:58,2.00000,2.00000,1.00000,2019-08-11 19:35:17,2019-08-11 19:35:17,Mens_Pants_Straight,2.00000,2.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59123842,It’s almost the end,Dockers_Adhoc_Promotional,20190811_US_Dockers_Tops_GraphicTees_30off_FS_...
1,273890682,2019-12-26,176031042,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-14 19:43:46,-1610606741,224451348.00000,111097,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59253142,"<#include ""cms://messagelibrary/email/59253142...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT4_Trigger_20191226
2,273890682,2019-12-26,176031082,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-12 18:43:46,1329572970,746908335.00000,175946,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59253042,"<#include ""cms://messagelibrary/email/59253042...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT3_Trigger_20191226
3,273890682,2019-12-26,176031062,Footer_Desktop,1,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-07 18:20:54,-2128439724,746908335.00000,163449,2.00000,2020-03-07 19:54:09,2020-03-07 20:47:41,1.00000,2.00000,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59251982,"<#include ""cms://messagelibrary/email/59251982...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT1_Trigger_20191226
4,273890682,2019-12-26,176031022,Footer_Desktop,0,0,27.74000,-72,3,6,Non-Holiday,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2020-03-10 17:43:46,2066955832,746908335.00000,228854,nan,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,NaN,nan,nan,59252822,"<#include ""cms://messagelibrary/email/59252822...",Dockers_Program_WelcomeGeneral,DOC_WelNoPurchT2_Trigger_20191226
5,260343962,2019-11-29,175893902,Lightbox_Mobile,1,0,59.98000,355,12,0,Non-Holiday,53,0,0,0,0,1,0,0,0,0,nan,nan,1.46535,0.01291,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,nan,nan,nan,nan,2019-11-29 13:07:15,230762889,nan,163400,2.00000,2019-11-29 16:30:49,2019-11-29 17:01:18,3.00000,4.00000,2.00000,2019-11-29 16:31:06,2019-11-29 17:01:35,"Mens, Sale",3.00000,4.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59216502,50% off absolutely everything + free shipping,Dockers_Adhoc_Promotional,20191129_US_Dockers_Dedicated_BF_50Off_FSFR_AM...
6,206326602,2019-11-29,175893902,My Account_Desktop,1,0,39.00000,1253,6,5,Non-Holiday,0,1,0,0,3,3,0,0,0,1,nan,nan,7.06333,1.05217,0,0,0,0,0,0,0,0,0,0,0,0,0,nan,160.00000,nan,nan,194.00000,2019-11-29 13:08:27,230762889,nan,164971,1.00000,2019-11-29 16:42:59,2019-11-29 16:42:59,3.00000,3.00000,3.00000,2019-11-29 16:43:02,2019-11-29 18:13:32,"Promotion, Mens_ShirtTops, Brand",3.00000,5.00000,nan,NaN,NaN,NaN,NaN,nan,nan,59216502

In [ ]:
df_10.info()

In [ ]:
df_50.info()

In [ ]:
df_100.info()

# For the chunks

| #  | Column                       | Chunk 1         | Chunk 10        | Chunk 50        | Chunk 100       | Dtype   | Description                                                          |
|----|------------------------------|-----------------|-----------------|-----------------|-----------------|---------|----------------------------------------------------------------------|
| 0  | riid                         | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Unique identifier of the user                                        |
| 1  | send_dt                      | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Date of sending the email                                            |
| 2  | launch_id                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Unique identifier of the campaign?                                   |
| 3  | sub_sourcedev                | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Subscriber Source/Device                                             |
| 4  | opened                       | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Whether opened or not                                                |
| 5  | unsub                        | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Whether unsubscriber or not                                          |
| 6  | rev_3dv2                     | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | float64 | Revenue (3-day timeframe from click?)                                |
| 7  | aq_dt                        | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Acquisition Date                                                     |
| 8  | aq_mo                        | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Acquisition Month                                                    |
| 9  | aq_dow                       | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Acquisition Day of Week                                              |
| 10 | aq_period                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Whether user acquired on holiday/not                                 |
| 11 | sends_since_last_open        | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days with email sends since last open                           |
| 12 | wk1_opens                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user opened an email in last 7 days                        |
| 13 | wk4_opens                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user opened an email in last 7 - 27 days                   |
| 14 | mo3_opens                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user opened an email in last 28 - 91 days                  |
| 15 | mo6_opens                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user opened an email in last 92 - 183 days                 |
| 16 | yr1_opens                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user opened an email in last 184 - 365 days                |
| 17 | wk1_clicks                   | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user clicked an email in last 7 days                       |
| 18 | wk4_clicks                   | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user clicked an email in last 7 - 27 days                  |
| 19 | mo3_clicks                   | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user clicked an email in last 28 - 91 days                 |
| 20 | mo6_clicks                   | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user clicked an email in last 92 - 183 days                |
| 21 | rev_dec                      | 32717 non-null  | 22767 non-null  | 23042 non-null  | 22989 non-null  | float64 | Total revenue for given user over past 2 years decayed exponentially |
| 22 | rev_raw                      | 32717 non-null  | 22767 non-null  | 23042 non-null  | 22989 non-null  | float64 | ?                                                                    |
| 23 | snds_dec                     | 78309 non-null  | 97613 non-null  | 97760 non-null  | 97683 non-null  | float64 | Total sends for given user over past 2 years decayed exponentially   |
| 24 | snds_opens_dec               | 78309 non-null  | 97613 non-null  | 97760 non-null  | 97683 non-null  | float64 | Total sends for given user over past 1 year decayed exponentially    |
| 25 | wk1_vis                      | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited site in past 7 days                           |
| 26 | wk4_vis                      | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited site in past 7 - 27 days                      |
| 27 | mo3_vis                      | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited site in past 28 - 91 days                     |
| 28 | mo6_vis                      | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited site in past 92 - 183 days                    |
| 29 | wk1_pv                       | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited product page on site in past 7 days           |
| 30 | wk4_pv                       | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited product page on site in past 7 - 27 days      |
| 31 | mo3_pv                       | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited product page on site in past 28 - 91 days     |
| 32 | mo6_pv                       | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of days user visited product page on site in past 92 - 183 days    |
| 33 | wk1_secs                     | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Total seconds user spent on site in past 7 days                      |
| 34 | wk4_secs                     | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Total seconds user spent on site in past 7-27 days                   |
| 35 | mo3_secs                     | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Total seconds user spent on site in past 28-91 days                  |
| 36 | mo6_secs                     | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Total seconds user spent on site in past 92-183 days                 |
| 37 | prev_optouts                 | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | # of times user has previously opted out                             |
| 38 | last_order                   | 32717 non-null  | 22767 non-null  | 23042 non-null  | 22989 non-null  | float64 | # of days since user placed last order                               |
| 39 | last_click                   | 44554 non-null  | 40711 non-null  | 40914 non-null  | 40643 non-null  | float64 | # of days since user clicked on email                                |
| 40 | last_cart                    | 288 non-null    | 130 non-null    | 141 non-null    | 122 non-null    | float64 | # of days since user engaged with cart on site                       |
| 41 | last_visit                   | 389 non-null    | 208 non-null    | 211 non-null    | 205 non-null    | float64 | # of days since user last visited site                               |
| 42 | last_ded_open                | 63781 non-null  | 78805 non-null  | 79421 non-null  | 79056 non-null  | float64 | # of days since user last opened a dedicated promo email             |
| 43 | sent_time                    | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Send time of the email                                               |
| 44 | offer_signature_id           | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Unique identifier of the offer?                                      |
| 45 | dynamic_content_signature_id | 20209 non-null  | 2119 non-null   | 2123 non-null   | 2287 non-null   | float64 | Unique identifier of the dynamic content?                            |
| 46 | message_size                 | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Message size (in B?)                                                 |
| 47 | open_cnt                     | 49570 non-null  | 21059 non-null  | 20996 non-null  | 21092 non-null  | float64 | # of times user opened this email                                    |
| 48 | first_open_time              | 49570 non-null  | 21059 non-null  | 20996 non-null  | 21092 non-null  | object  | First time user opened the email                                     |
| 49 | last_open_time               | 49570 non-null  | 21059 non-null  | 20996 non-null  | 21092 non-null  | object  | Last time user opened the email                                      |
| 50 | hrs_to_first_open            | 49570 non-null  | 21059 non-null  | 20996 non-null  | 21092 non-null  | float64 | # of hours since user opened email for first time                    |
| 51 | hrs_to_last_open             | 49570 non-null  | 21059 non-null  | 20996 non-null  | 21092 non-null  | float64 | # of hours since user opened email for last time                     |
| 52 | click_cnt                    | 28824 non-null  | 1449 non-null   | 1417 non-null   | 1477 non-null   | float64 | # of times user clicked on email                                     |
| 53 | first_click_time             | 28824 non-null  | 1449 non-null   | 1417 non-null   | 1477 non-null   | object  | First time user clicked on email                                     |
| 54 | last_click_time              | 28824 non-null  | 1449 non-null   | 1417 non-null   | 1477 non-null   | object  | Last time user clicked on email                                      |
| 55 | offer_category               | 28710 non-null  | 1445 non-null   | 1413 non-null   | 1474 non-null   | object  | Categories that offer falls into                                     |
| 56 | hrs_to_first_click           | 28824 non-null  | 1449 non-null   | 1417 non-null   | 1477 non-null   | float64 | # of hours since user clicked on email for first time                |
| 57 | hrs_to_last_click            | 28824 non-null  | 1449 non-null   | 1417 non-null   | 1477 non-null   | float64 | # of hours since user clicked on email for last time                 |
| 58 | opt_out_cnt                  | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | float64 | # of times user opted out?                                           |
| 59 | first_opt_out_time           | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | object  | First time user opted out?                                           |
| 60 | last_opt_out_time            | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | object  | Last time user opted out?                                            |
| 61 | opt_out_source               | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | object  | How user opted out?                                                  |
| 62 | opt_out_reason               | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | object  | Reason for opting out                                                |
| 63 | hrs_to_first_opt_out         | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | float64 | # of hours since user opted out for first time                       |
| 64 | hrs_to_last_opt_out          | 520 non-null    | 293 non-null    | 268 non-null    | 292 non-null    | float64 | # of hours since user opted out for last time                        |
| 65 | campaign_id                  | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | int64   | Unique identifier of the campaign                                    |
| 66 | subject                      | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Subject line of the email                                            |
| 67 | marketing_strategy           | 97208 non-null  | 95476 non-null  | 95429 non-null  | 95386 non-null  | object  | Type of marketing strategy                                           |
| 68 | campaign_type                | 100000 non-null | 100000 non-null | 100000 non-null | 100000 non-null | object  | Internal ID for which campaign                                       |